In [ ]:
import os
import time
import joblib
import numpy as np
from copy import deepcopy
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
data = load_breast_cancer()
X = data.data.astype(np.float32, copy=False)
y = data.target.astype(np.int32, copy=False)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("Classes:", np.unique(y_test))

In [ ]:
MODEL_PATH = r"logreg_model_before_optimization.joblib"

def _is_fitted_logreg(m):
    return hasattr(m, "coef_") and hasattr(m, "intercept_") and hasattr(m, "classes_")

if os.path.exists(MODEL_PATH):
    obj = joblib.load(MODEL_PATH)

    if not isinstance(obj, LogisticRegression):
        raise TypeError(f"Loaded object is not LogisticRegression. Type={type(obj)}")

    if not _is_fitted_logreg(obj):
        raise ValueError("Loaded LogisticRegression is not fitted (missing coef_/intercept_/classes_).")

    model = obj
    print("Loaded LogisticRegression from:", MODEL_PATH)

else:
    model = LogisticRegression(
        penalty="l2",
        C=1.0,
        solver="lbfgs",
        max_iter=2000,
        n_jobs=None
    )
    model.fit(X_train, y_train)
    print("Trained baseline LogisticRegression (because file not found).")

print("Model ready. Fitted:", _is_fitted_logreg(model))

In [ ]:
def export_joblib(m, path: str) -> float:
    joblib.dump(m, path)
    return os.path.getsize(path) / (1024 * 1024)

p = model.predict_proba(X_test)[:, 1]
y_hat = (p >= 0.5).astype(np.int32)
acc = float(accuracy_score(y_test, y_hat))

base_path = "logreg_baseline.joblib"
base_mb = export_joblib(model, base_path)

print("Baseline accuracy:", acc)
print("Baseline exported:", base_path, f"({base_mb:.6f} MB)")

In [ ]:
PRUNE_ABS_WEIGHT_BELOW = 1e-4 

optimized_model = deepcopy(model)

optimized_model.coef_ = optimized_model.coef_.astype(np.float32, copy=False)
optimized_model.intercept_ = optimized_model.intercept_.astype(np.float32, copy=False)

if PRUNE_ABS_WEIGHT_BELOW > 0:
    w = optimized_model.coef_
    mask = np.abs(w) < PRUNE_ABS_WEIGHT_BELOW
    w = w.copy()
    w[mask] = 0.0
    optimized_model.coef_ = w

print("Optimization done.")
print("Prune threshold:", PRUNE_ABS_WEIGHT_BELOW)

In [ ]:
p_opt = optimized_model.predict_proba(X_test)[:, 1]
y_hat_opt = (p_opt >= 0.5).astype(np.int32)
acc_opt = float(accuracy_score(y_test, y_hat_opt))

opt_path = "logreg_optimized.joblib"
opt_mb = export_joblib(optimized_model, opt_path)

loaded = joblib.load(opt_path)
if not isinstance(loaded, LogisticRegression) or not _is_fitted_logreg(loaded):
    raise ValueError("Reload failed or loaded model is not a fitted LogisticRegression.")

p_loaded = loaded.predict_proba(X_test)[:, 1]
max_diff = float(np.max(np.abs(p_loaded - p_opt)))

print("Optimized accuracy:", acc_opt)
print("Optimized exported:", opt_path, f"({opt_mb:.6f} MB)")
print("Reload max |diff|:", max_diff)

In [ ]:
def _sigmoid(x: np.ndarray) -> np.ndarray:
    return 1.0 / (1.0 + np.exp(-x))

def measure_inference_ms_logreg(
    m: LogisticRegression,
    X: np.ndarray,
    batch_size: int = 1,
    n_warmup: int = 50,
    n_runs: int = 500,
) -> dict:
    if not _is_fitted_logreg(m):
        raise ValueError("Model is not fitted.")
    X = np.asarray(X, dtype=np.float32)
    n = X.shape[0]
    if n == 0:
        raise ValueError("X is empty.")
    if batch_size <= 0:
        raise ValueError("batch_size must be >= 1.")

    starts = [((i * batch_size) % n) for i in range(max(n_warmup, n_runs))]
    batches = [X[s:s+batch_size] for s in starts]

    for xb in batches[:n_warmup]:
        _ = m.predict_proba(xb)

    t_sklearn = []
    for xb in batches[:n_runs]:
        t0 = time.perf_counter()
        _ = m.predict_proba(xb)
        t1 = time.perf_counter()
        t_sklearn.append((t1 - t0) * 1000.0)

    t_sklearn = np.asarray(t_sklearn, dtype=np.float64)

    w = m.coef_.astype(np.float32, copy=False)
    b = m.intercept_.astype(np.float32, copy=False)

    for xb in batches[:n_warmup]:
        z = xb @ w.T + b
        _ = _sigmoid(z)

    t_numpy = []
    for xb in batches[:n_runs]:
        t0 = time.perf_counter()
        z = xb @ w.T + b
        _ = _sigmoid(z)
        t1 = time.perf_counter()
        t_numpy.append((t1 - t0) * 1000.0)

    t_numpy = np.asarray(t_numpy, dtype=np.float64)

    def pack(arr):
        return {
            "mean_ms_per_batch": float(arr.mean()),
            "p50_ms_per_batch": float(np.percentile(arr, 50)),
            "p95_ms_per_batch": float(np.percentile(arr, 95)),
        }

    sk = pack(t_sklearn)
    npk = pack(t_numpy)

    return {
        "batch_size": int(batch_size),
        "sklearn": {
            **sk,
            "mean_ms_per_sample": sk["mean_ms_per_batch"] / batch_size,
            "p50_ms_per_sample": sk["p50_ms_per_batch"] / batch_size,
            "p95_ms_per_sample": sk["p95_ms_per_batch"] / batch_size,
        },
        "numpy": {
            **npk,
            "mean_ms_per_sample": npk["mean_ms_per_batch"] / batch_size,
            "p50_ms_per_sample": npk["p50_ms_per_batch"] / batch_size,
            "p95_ms_per_sample": npk["p95_ms_per_batch"] / batch_size,
        },
    }

In [ ]:
batch_sizes = [1, 8, 32, 128]
results = []

for bs in batch_sizes:
    rb = measure_inference_ms_logreg(model, X_test, batch_size=bs)
    rb["model"] = "baseline"
    results.append(rb)

    ro = measure_inference_ms_logreg(optimized_model, X_test, batch_size=bs)
    ro["model"] = "optimized"
    results.append(ro)

for bs in batch_sizes:
    b = next(r for r in results if r["model"] == "baseline" and r["batch_size"] == bs)
    o = next(r for r in results if r["model"] == "optimized" and r["batch_size"] == bs)

    print(f"\nBatch size = {bs}")
    print(f"  Baseline  sklearn: mean {b['sklearn']['mean_ms_per_batch']:.4f} ms/batch | {b['sklearn']['mean_ms_per_sample']:.4f} ms/sample")
    print(f"  Optimized sklearn: mean {o['sklearn']['mean_ms_per_batch']:.4f} ms/batch | {o['sklearn']['mean_ms_per_sample']:.4f} ms/sample")

    print(f"  Baseline  numpy  : mean {b['numpy']['mean_ms_per_batch']:.4f} ms/batch | {b['numpy']['mean_ms_per_sample']:.4f} ms/sample")
    print(f"  Optimized numpy  : mean {o['numpy']['mean_ms_per_batch']:.4f} ms/batch | {o['numpy']['mean_ms_per_sample']:.4f} ms/sample")

In [ ]:
import time
import numpy as np
from sklearn.linear_model import LogisticRegression

def measure_inference_time_logistic(
    model: LogisticRegression,
    X: np.ndarray,
    num_runs: int = 100,
    warm_up: int = 10,
    use_proba: bool = True
):
    if not isinstance(model, LogisticRegression):
        raise TypeError(f"model must be sklearn.linear_model.LogisticRegression, got: {type(model)}")

    if not (hasattr(model, "coef_") and hasattr(model, "intercept_") and hasattr(model, "classes_")):
        raise ValueError("LogisticRegression model is not fitted (missing coef_/intercept_/classes_).")

    X = np.asarray(X, dtype=np.float32)
    if X.ndim != 2:
        raise ValueError(f"X must be 2D array [n_samples, n_features], got shape: {X.shape}")
    if X.shape[0] == 0:
        raise ValueError("X has 0 rows.")

    if num_runs <= 0:
        raise ValueError("num_runs must be >= 1.")
    if warm_up < 0:
        raise ValueError("warm_up must be >= 0.")

    if use_proba:
        if not hasattr(model, "predict_proba"):
            raise AttributeError("model has no predict_proba(). Set use_proba=False.")
        infer = model.predict_proba
    else:
        infer = model.predict

    for _ in range(warm_up):
        _ = infer(X)

    start = time.perf_counter()
    for _ in range(num_runs):
        _ = infer(X)
    end = time.perf_counter()

    total_time_sec = end - start
    avg_time_per_run_sec = total_time_sec / num_runs
    num_samples = X.shape[0]

    avg_time_per_sample_us = (avg_time_per_run_sec / num_samples) * 1e6
    total_avg_time_ms = avg_time_per_run_sec * 1000

    print(f"--- Inference Speed ({num_runs} runs) ---")
    print(f"Total time per batch: {total_avg_time_ms:.4f} ms")
    print(f"Time per sample:      {avg_time_per_sample_us:.4f} µs")

    return {
        "time_per_sample_us": float(avg_time_per_sample_us),
        "total_time_ms": float(total_avg_time_ms),
        "num_samples": int(num_samples),
        "use_proba": bool(use_proba),
    }

In [ ]:
print("Baseline LogisticRegression:")
_ = measure_inference_time_logistic(model, X_test)

print("\nOptimized LogisticRegression:")
_ = measure_inference_time_logistic(optimized_model, X_test)

In [ ]:
def export_logreg_to_h(m: LogisticRegression, out_path: str) -> bool:
    try:
        import m2cgen as m2c
        import re
    except Exception as e:
        print("[WARN] m2cgen not available:", e)
        return False

    if not _is_fitted_logreg(m):
        raise ValueError("Model is not fitted.")

    try:
        c_code = m2c.export_to_c(m)

        c_code = re.sub(
            r"memcpy\(([^,]+),\s*\((?:double|float)\[\]\)\{([^}]+)\},\s*(\d+)\s*\*\s*sizeof\((double|float)\)\);",
            r"{ \4 tmp[] = {\2}; memcpy(\1, tmp, \3 * sizeof(\4)); }",
            c_code
        )

        stem = os.path.splitext(os.path.basename(out_path))[0]
        header_guard = stem.upper() + "_H"

        wrapped = (
            f"#ifndef {header_guard}\n#define {header_guard}\n\n"
            f"{c_code}\n\n"
            f"#endif // {header_guard}\n"
        )

        with open(out_path, "w", encoding="utf-8") as f:
            f.write(wrapped)

        print("[INFO] Exported header:", out_path)
        print(f"[HINT] Include using: #include \"{os.path.basename(out_path)}\"")
        return True

    except Exception as e:
        print("[WARN] Header export failed:", e)
        return False

export_logreg_to_h(optimized_model, "logreg_optimized.h")

In [ ]:
def coef_stats(m: LogisticRegression):
    w = np.asarray(m.coef_)
    total = w.size
    nonzero = int(np.count_nonzero(w))
    zero = total - nonzero
    return {
        "total_weights": int(total),
        "nonzero_weights": int(nonzero),
        "zero_weights": int(zero),
        "sparsity_%": float(100.0 * zero / total) if total else 0.0
    }

print("Baseline stats :", coef_stats(model))
print("Optimized stats:", coef_stats(optimized_model))

if os.path.exists("logreg_baseline.joblib") and os.path.exists("logreg_optimized.joblib"):
    b = os.path.getsize("logreg_baseline.joblib") / (1024 * 1024)
    o = os.path.getsize("logreg_optimized.joblib") / (1024 * 1024)
    print(f"File size baseline : {b:.6f} MB")
    print(f"File size optimized: {o:.6f} MB")

In [ ]:
def evaluate_overfitting(model, X_train, y_train, X_test, y_test, name="model"):
    p_train = model.predict_proba(X_train)[:, 1]
    p_test  = model.predict_proba(X_test)[:, 1]

    acc_train = accuracy_score(y_train, (p_train >= 0.5).astype(int))
    acc_test  = accuracy_score(y_test,  (p_test  >= 0.5).astype(int))

    gap = acc_train - acc_test

    print(f"[{name}] Train acc: {acc_train:.4f}")
    print(f"[{name}] Test  acc: {acc_test:.4f}")
    print(f"[{name}] Gap       : {gap:.4f}")

    return {
        "train_acc": float(acc_train),
        "test_acc": float(acc_test),
        "gap": float(gap)
    }

print("Baseline:")
_ = evaluate_overfitting(model, X_train, y_train, X_test, y_test, name="baseline")

print("\nOptimized:")
_ = evaluate_overfitting(optimized_model, X_train, y_train, X_test, y_test, name="optimized")